In [4]:
import pandas as pd
#import pymssql as sql
import pyodbc as pod

In [5]:
Shipping = pd.read_json('D:/PEI Data Analysis task/Shipping.json')
Order = pd.read_csv('D:/PEI Data Analysis task/Order.csv')
Customer = pd.read_excel('D:/PEI Data Analysis task/Customer.xls')

In [6]:
Shipping.head()

,Shipping_ID,Status,Customer_ID
0,1,Pending,173
1,2,Pending,155
2,3,Delivered,242
3,4,Pending,223
4,5,Delivered,72


In [7]:
Order.head()

,Order_ID,Item,Amount,Customer_ID
0,1,Keyboard,400,139
1,2,Mouse,300,250
2,3,Monitor,12000,239
3,4,Keyboard,400,153
4,5,Mousepad,250,153


In [8]:
Customer.head()

,Customer_ID,First,Last,Age,Country
0,1,Joseph,Rice,43,USA
1,2,Gary,Moore,71,USA
2,3,John,Walker,44,UK
3,4,Eric,Carter,38,UK
4,5,William,Jackson,58,UAE


Missing date columns on orders and shipping.
Shipping doesnt have OrderID to check the status of the order. One customer may have more than 1 order.

In [9]:
#Checking for NULL values on the datasets
print('\r\n-------\r\nCustomer\r\n-------\r\n',Customer.isnull().sum())
print('\r\n-------\r\nOrder\r\n-------\r\n',Order.isnull().sum())
print('\r\n-------\r\nShipping\r\n-------\r\n',Shipping.isnull().sum())

# No null values observed


-------
Customer
-------
 Customer_ID    0
First          0
Last           0
Age            0
Country        0
dtype: int64

-------
Order
-------
 Order_ID       0
Item           0
Amount         0
Customer_ID    0
dtype: int64

-------
Shipping
-------
 Shipping_ID    0
Status         0
Customer_ID    0
dtype: int64


In [12]:
def ins_conn_db(df,table_name):
    server = 'DESKTOP-JAPNGMO'
    database = 'DataAnalytics'
    conn = pod.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')
    try:
        conn = conn.cursor()
        column = ', '.join(df.columns)
        placeholder =', '.join(['?'] * len(df.columns))
        conn = conn.execute(f'TRUNCATE TABLE {table_name}')
        conn.commit()
        insert_sql = f'INSERT INTO dbo.[{table_name}] ({column}) VALUES({placeholder})'
        conn.executemany(insert_sql, df.values.tolist())
        print('Connection Success')
    except:
        print('Connection Failed')
    conn.commit()
    conn.close()

In [13]:
ins_conn_db(Shipping,'PEI_Shipping')

Connection Success


In [14]:
ins_conn_db(Customer,'PEI_Customer')

Connection Success


In [15]:
ins_conn_db(Order,'PEI_Order')

Connection Success


In [16]:
#connecting to DB to read data from SQL

def read_conn_db(query):
    server = 'DESKTOP-JAPNGMO'
    database = 'DataAnalytics'
    conn = pod.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')
    try:
        result = pd.read_sql_query(query,conn)
        print('Connection Success')
    except:
        print('Connection Failed')
    conn.close()
    return result 
    

In [17]:
query = '''WITH Cust AS
(SELECT 
	Customer_ID
	,First+' '+Last as [Full Name]
	,CASE WHEN Age <= 30 THEN 'Below 30' ELSE 'Above 30' end as [Age Group] 
	,Country
FROM dbo.PEI_Customer
)

,Valid_Ship AS 
(
SELECT 
	DISTINCT [Status]
	,Customer_ID
	,ROW_NUMBER() OVER (PARTITION BY Customer_ID ORDER BY Status) as RNM
FROM dbo.PEI_Shipping
WHERE Customer_ID IN (SELECT Customer_ID FROM dbo.PEI_Order)
)

SELECT 
	CAST(Order_ID AS INT) AS Order_ID
	,CAST(Item AS nvarchar(15)) AS Item
	,CAST([Full Name] AS nvarchar(50)) AS [FullName]
	,CAST([Age Group] AS nvarchar(10)) AS [AgeGroup]
	,CAST(Country AS nvarchar(5)) AS [Country]
	,CAST((CASE WHEN [Status] IS NULL THEN 'Unknown' ELSE [Status] END) AS nvarchar(15)) AS [Status]
	,CAST(Amount AS INT) AS Amount 
FROM dbo.PEI_Order O
	LEFT JOIN Cust C
		ON C.Customer_ID = o.Customer_ID
	LEFT JOIN Valid_Ship S
		ON O.Customer_ID = s.Customer_ID
		AND S.RNM = 1
Order by Order_ID

'''

result = read_conn_db(query)


Connection Success


C:\Users\DELL\AppData\Local\Temp\ipykernel_8768\3104063460.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query,conn)


In [52]:
result

,Order_ID,Item,FullName,AgeGroup,Country,Status,Amount
0,1,Keyboard,Ryan Martin,Above 30,UK,Unknown,400
1,2,Mouse,Stephen Jones,Below 30,USA,Unknown,300
2,3,Monitor,Janet Holmes,Above 30,UK,Unknown,12000
3,4,Keyboard,Janet Valdez,Below 30,UK,Unknown,400
4,5,Mousepad,Janet Valdez,Below 30,UK,Unknown,250
...,...,...,...,...,...,...,...
245,246,Harddisk,Xavier Miles,Above 30,USA,Pending,5000
246,247,Webcam,Shawn Johnson,Below 30,UAE,Delivered,350
247,248,Mousepad,Tiffany Pearson,Above 30,UAE,Delivered,200
248,249,DDR RAM,Tiffany Bowers,Above 30,USA,Unknown,1500


In [18]:
result.to_csv("D:/PEI Data Analysis task/Combined_Dataset.csv",)